In [11]:
!pip install keras-cv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.6/721.6 kB 28.3 MB/s eta 0:00:00


In [12]:
import os
import cv2
from keras.applications import Xception
from keras.layers import Dense, Flatten, Dropout
from keras.models import Sequential
import pandas as pd
from keras.utils import image_dataset_from_directory
from keras_cv.layers.preprocessing import RandomSaturation

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf

In [4]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5



In [18]:
model = Sequential()
model.add(RandomSaturation(factor=(0.75, 0.75)))
model.add(Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3), classes=2))
model.add(Flatten())
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(rate=0.4))
model.add(Dense(128, activation='sigmoid'))
model.add(Dropout(rate=0.2))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(rate=0.1))
model.add(Dense(2, activation='softmax'))

In [25]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_saturation_1 (Random  (None, 224, 224, 3)      0         
 Saturation)                                                     
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 flatten_3 (Flatten)         (None, 100352)            0         
                                                                 
 dense_9 (Dense)             (None, 256)               25690368  
                                                                 
 dropout_6 (Dropout)         (None, 256)               0         
                                                                 
 dense_10 (Dense)            (None, 128)               32896     
                                                      

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [14]:
train_ds = image_dataset_from_directory(
    directory='/content/drive/My Drive/IndoorOutdoorClassification/images',
    labels='inferred',
    label_mode='categorical',
    image_size=(224, 224),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='training'
)
val_ds = image_dataset_from_directory(
    directory='/content/drive/My Drive/IndoorOutdoorClassification/images',
    labels='inferred',
    label_mode='categorical',
    image_size=(224, 224),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='validation'
)

Found 800 files belonging to 2 classes.
Using 640 files for training.
Found 800 files belonging to 2 classes.
Using 160 files for validation.


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE



Found 800 files belonging to 2 classes.
Using 160 files for validation.


In [79]:
# 200
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10
20/20 [==============================] - 12s 562ms/step - loss: 1.0510e-04 - accuracy: 1.0000 - recall_3: 1.0000 - precision_3: 1.0000 - val_loss: 0.4521 - val_accuracy: 0.9187 - val_recall_3: 0.9187 - val_precision_3: 0.9187
Epoch 2/10
20/20 [==============================] - 14s 647ms/step - loss: 1.0717e-04 - accuracy: 1.0000 - recall_3: 1.0000 - precision_3: 1.0000 - val_loss: 0.4509 - val_accuracy: 0.9250 - val_recall_3: 0.9250 - val_precision_3: 0.9250
Epoch 3/10
20/20 [==============================] - 14s 651ms/step - loss: 1.2437e-04 - accuracy: 1.0000 - recall_3: 1.0000 - precision_3: 1.0000 - val_loss: 0.4484 - val_accuracy: 0.9250 - val_recall_3: 0.9250 - val_precision_3: 0.9250
Epoch 4/10
20/20 [==============================] - 13s 593ms/step - loss: 1.1295e-04 - accuracy: 1.0000 - recall_3: 1.0000 - precision_3: 1.0000 - val_loss: 0.4464 - val_accuracy: 0.9250 - val_recall_3: 0.9250 - val_precision_3: 0.9250
Epoch 5/10
20/20 [==============================] - 

In [81]:
save_directory='/content/drive/My Drive/IndoorOutdoorClassification/xception_saturate_200e.h5'
model.save(save_directory)

## Evaluation

In [22]:
import numpy as np

In [23]:
def predict_image_softmax(model, path: str,  size_x: int = 224, size_y: int = 224):
  img = cv2.imread(path)
  img = cv2.resize(img, (size_x, size_y))
  img = np.expand_dims(img, axis=0)
  return model.predict(img, verbose=False)[0]

In [67]:
prediction_dir = '/content/drive/My Drive/IndoorOutdoorClassification/images/indoor'
cnt_zero = 0
cnt_one = 0
imgs = os.listdir(prediction_dir)

for img in imgs:
  try:
    prediction = predict_image_softmax(model,f'{prediction_dir}/{img}')
    # print(f'{img} - {prediction}')
    if(prediction[0]>=prediction[1]):
      cnt_zero += 1
    else:
      cnt_one += 1
  except Exception:
    print('except')
print(f"Class zero: {cnt_zero} \n Class one: {cnt_one}")

Class zero: 343 
 Class one: 57


In [66]:
prediction_dir = '/content/drive/My Drive/IndoorOutdoorClassification/images/outdoor'
cnt_zero = 0
cnt_one = 0
imgs = os.listdir(prediction_dir)

for img in imgs:
  try:
    prediction = predict_image_softmax(model,f'{prediction_dir}/{img}')
    # print(f'{img} - {prediction}')
    if(prediction[0]>=prediction[1]):
      cnt_zero += 1
    else:
      cnt_one += 1
  except Exception:
    print('except')
print(f"Class zero: {cnt_zero} \n Class one: {cnt_one}")

Class zero: 30 
 Class one: 370


In [80]:
prediction_dir = '/content/drive/My Drive/IndoorOutdoorClassification/prediction_images/'
cnt_zero = 0
cnt_one = 0
imgs = os.listdir(prediction_dir)
imgs = sorted(imgs)
for img in imgs:
  try:
    prediction = predict_image_softmax(model,f'{prediction_dir}/{img}')
    print(f'{img} - {prediction} - {"indoor" if prediction[0]>=prediction[1] else "outdoor"}')
    if(prediction[0]>=prediction[1]):
      cnt_zero += 1
    else:
      cnt_one += 1
  except Exception:
    print('except')
print(f"Class zero: {cnt_zero} \n Class one: {cnt_one}")

indoor1.jpg - [0.51319015 0.48680988] - indoor
indoor2.jpg - [0.9900407  0.00995934] - indoor
indoor3.jpg - [0.3811899  0.61881006] - outdoor
indoor4.jpg - [9.9990773e-01 9.2235794e-05] - indoor
outdoor1.jpg - [6.3341875e-05 9.9993670e-01] - outdoor
outdoor2.jpg - [9.9974424e-01 2.5574345e-04] - indoor
outdoor3.jpg - [3.4977973e-04 9.9965024e-01] - outdoor
outdoor4.jpg - [0.03097816 0.96902186] - outdoor
Class zero: 4 
 Class one: 4
